# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:

cities_df = pd.read_csv('../output_data/cities_t.csv')
cities_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Bubaque,11.2833,-15.8333,76.10,73,81,5.93,GW,1618718831
1,Chippewa Falls,44.9369,-91.3929,50.00,40,75,3.44,US,1618718402
2,Lebu,-37.6167,-73.6500,55.24,88,100,7.27,CL,1618718398
3,Lakes Entrance,-37.8811,147.9810,68.00,53,8,3.00,AU,1618718832
4,Atuona,-9.8000,-139.0333,80.74,74,36,18.95,PF,1618718348
...,...,...,...,...,...,...,...,...,...
559,Misratah,32.3754,15.0925,60.66,77,100,9.60,LY,1618719005
560,Cleveland,41.4995,-81.6954,39.20,81,40,5.75,US,1618718882
561,Nkhotakota,-12.9274,34.2961,69.17,83,79,12.75,MW,1618719005
562,Churapcha,62.0025,132.4325,25.47,64,100,12.91,RU,1618719006


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)
locations = cities_df[['Lat', 'Lng']].astype(float)
humidity = cities_df['Humidity'].astype(float)

In [4]:
# Create a poverty Heatmap layer
figure_layout = {
    'width': '1000px',
    'height': '700px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
    }
fig = gmaps.figure(figure_layout)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [11]:
cities_temp_df = cities_df[(cities_df['Max Temp'] >= 70) & (cities_df['Max Temp'] <= 80)]
cities_temp_ws_df = cities_temp_df[(cities_temp_df['Wind Speed'] < 10) & (cities_temp_df['Cloudiness'] == 0)]
# cities_temp_df.to_csv('../output_data/cities_maxtemp.csv', index=False)
cities_temp_ws_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
33,Cape Town,-33.9258,18.4232,73.99,76,0,1.14,ZA,1618718213
37,Busselton,-33.6500,115.3333,75.99,48,0,1.99,AU,1618718379
66,Hermanus,-34.4187,19.2345,73.99,19,0,8.21,ZA,1618718339
88,Salalah,17.0151,54.0924,77.00,50,0,3.44,OM,1618718335
97,Esperance,-33.8667,121.9000,71.60,46,0,4.61,AU,1618718858
159,Boa Vista,2.8197,-60.6733,78.80,78,0,5.75,BR,1618718668
163,Acapulco de Juárez,16.8634,-99.8901,77.13,70,0,7.87,MX,1618718598
311,Etchoropo,26.7667,-109.6667,73.99,52,0,6.69,MX,1618718923
329,Muhafazat Lahij,13.1667,44.5833,73.15,70,0,6.40,YE,1618718928
403,Baracoa,20.3467,-74.4958,75.31,80,0,5.46,CU,1618718952


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [12]:
hotel_df = cities_temp_ws_df.reset_index()
hotel_df

,index,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,33,Cape Town,-33.9258,18.4232,73.99,76,0,1.14,ZA,1618718213
1,37,Busselton,-33.6500,115.3333,75.99,48,0,1.99,AU,1618718379
2,66,Hermanus,-34.4187,19.2345,73.99,19,0,8.21,ZA,1618718339
3,88,Salalah,17.0151,54.0924,77.00,50,0,3.44,OM,1618718335
4,97,Esperance,-33.8667,121.9000,71.60,46,0,4.61,AU,1618718858
5,159,Boa Vista,2.8197,-60.6733,78.80,78,0,5.75,BR,1618718668
6,163,Acapulco de Juárez,16.8634,-99.8901,77.13,70,0,7.87,MX,1618718598
7,311,Etchoropo,26.7667,-109.6667,73.99,52,0,6.69,MX,1618718923
8,329,Muhafazat Lahij,13.1667,44.5833,73.15,70,0,6.40,YE,1618718928
9,403,Baracoa,20.3467,-74.4958,75.31,80,0,5.46,CU,1618718952


In [13]:
hotel_df['Hotel Name'] = ''
# params dictionary to update each iteration
params = {
    "location": '25.0213, 37.2685',
    "radius": 100000,
    "types": "Hotel",
    "keyword": "hotel",
    "key": g_key
}

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
hotel = requests.get(base_url, params=params).json()
hotel["results"][0]["name"]




'Royal Tours Permanent Camp'

In [14]:
hotel_df['Hotel Name'] = ''
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "Hotel",
    "keyword": "hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    
    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

        # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    hotel = requests.get(base_url, params=params).json()
   
    try:
        hotel_df.loc[index, 'Hotel Name'] = hotel["results"][0]["name"]
    except:
        print(f'Missing cities {lat}, {lng}')
hotel_df

Missing cities 26.7667, -109.6667
Missing cities 13.1667, 44.5833
Missing cities 20.5169, -13.0499


,index,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,33,Cape Town,-33.9258,18.4232,73.99,76,0,1.14,ZA,1618718213,Atlanticview Cape Town Boutique Hotel
1,37,Busselton,-33.6500,115.3333,75.99,48,0,1.99,AU,1618718379,The Sebel Busselton
2,66,Hermanus,-34.4187,19.2345,73.99,19,0,8.21,ZA,1618718339,Harbour House Hotel
3,88,Salalah,17.0151,54.0924,77.00,50,0,3.44,OM,1618718335,HYATT SALALAH HOTEL
4,97,Esperance,-33.8667,121.9000,71.60,46,0,4.61,AU,1618718858,The Jetty Resort
5,159,Boa Vista,2.8197,-60.6733,78.80,78,0,5.75,BR,1618718668,Hotel Orla do Rio Branco
6,163,Acapulco de Juárez,16.8634,-99.8901,77.13,70,0,7.87,MX,1618718598,Dreams Acapulco Resort & Spa
7,311,Etchoropo,26.7667,-109.6667,73.99,52,0,6.69,MX,1618718923,
8,329,Muhafazat Lahij,13.1667,44.5833,73.15,70,0,6.40,YE,1618718928,
9,403,Baracoa,20.3467,-74.4958,75.31,80,0,5.46,CU,1618718952,Hotel Porto Santo


In [15]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
hotel_info
locations = hotel_df[["Lat", "Lng"]]

In [16]:
# Add marker layer ontop of heat map
locations1 = cities_df[['Lat', 'Lng']].astype(float)
humidity1 = cities_df['Humidity'].astype(float)
# hotel1 = hotel_df('Hotel Name').astype(str)
figure_layout = {
    'width': '1000px',
    'height': '700px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
    }
fig = gmaps.figure(figure_layout)

heat_layer = gmaps.heatmap_layer(locations1, weights=humidity1)
marker = gmaps.marker_layer(locations, info_box_content=[f"Poverty Rate: {hotel}" for hotel in hotel_info])

fig.add_layer(marker)
fig.add_layer(heat_layer)
fig

# Display figure


Figure(layout=FigureLayout(height='420px'))